## <b>3.3 PREPROCESSAMENT I ANÀLISI DE DADES</b>

### <b>3.3.2 Transformacions i enginyeria de variables bàsica</b>

#### <b>3.3.2.1 Preparació inicial i tractament de valors nuls</b>

In [4]:
# ============================================================
# 3.3.2.1 Preparació inicial i tractament de valors nuls
# ============================================================
# Objectiu d'aquest script:
#   - Carrego el dataset net (clean_motor_insurance.csv) com a punt de partida
#     per iniciar la fase d’enginyeria bàsica.
#   - Verifico que el conjunt de dades tingui integritat mínima abans de tocar res:
#        * dimensions (files, columnes)
#        * tipus de dades
#        * duplicats
#        * valors nuls
#   - Tracto els valors nuls segons el seu significat (no tots els NaN volen dir "dada perduda"):
#        * Nuls estructurals (Date_lapse):
#             - Creo Has_lapse (1 si hi ha baixa, 0 si no)
#             - Creo Is_active (1 si actiu, 0 si no)
#             - Derivo Policy_duration (només per pòlisses amb lapse)
#        * Nuls moderats (Length ~9.79%):
#             - Creo Length_missing_flag (per saber si he imputat o no)
#             - Imputo per mediana segmentada per Type_risk
#               (i si un segment no té valors, faig servir la mediana global com a pla B)
#        * Nuls baixos (Type_fuel ~1.67%):
#             - Imputo amb la categoria 'Unknown' (la codificació la deixo per més endavant)
#
# Nota:
#   - En aquest subapartat encara NO genero les derivades temporals finals
#     (Driver_age, Vehicle_age, etc.) ni faig codificació avançada.
#   - El resultat és un dataframe `df_prep` sense nuls no estructurals,
#     i amb algunes variables bàsiques de control i qualitat.
# ============================================================

import pandas as pd
import numpy as np

# ------------------------------------------------------------
# 1) Càrrega del dataset net
# ------------------------------------------------------------
# Defineixo la ruta del fitxer net que ja ve de la fase anterior.
CLEAN_PATH = "clean_motor_insurance.csv"  # Ruta del fitxer net previ

# Llegeixo el CSV de manera robusta deixant que pandas infereixi el separador.
# Això em va bé si el fitxer ve amb ',' o amb ';' (o canvia segons exportació).
df_prep = pd.read_csv(CLEAN_PATH, sep=None, engine="python")

# Confirmo per pantalla que la càrrega s'ha fet bé i miro dimensions per tenir un primer control.
print("clean_motor_insurance.csv carregat correctament")
print("Dimensions:", df_prep.shape)  # (n_files, n_columnes)

# ------------------------------------------------------------
# 2) Verificacions bàsiques d'integritat
# ------------------------------------------------------------
# Aquí faig comprovacions ràpides per detectar problemes típics:
# duplicats exactes i volum de nuls per variable.

# 2.1 Duplicats exactes (files)
# .duplicated() marca True per files idèntiques a una anterior; amb .sum() en compto el total.
n_dup = df_prep.duplicated().sum()
print("\nDuplicats exactes:", n_dup)

# 2.2 Taula de nuls inicial
# Calculo nombre i percentatge de nuls per columna per saber on he de centrar el tractament.
missing_count = df_prep.isna().sum()
missing_pct = (missing_count / len(df_prep) * 100).round(2)

# Em construeixo una taula ordenada per veure primer les variables amb més nuls.
missing_table = (
    pd.DataFrame({"missing_count": missing_count, "missing_pct": missing_pct})
      .sort_values("missing_count", ascending=False)
)

print("\nTaula de nuls inicial (top 10):")
# `display` m'ajuda si estic en notebook perquè es vegi en format taula.
display(missing_table.head(10))

# ------------------------------------------------------------
# 3) Tractament de nuls estructurals (Date_lapse)
# ------------------------------------------------------------
# Aquí el punt clau és que `Date_lapse` pot ser NaN perquè la pòlissa segueix activa.
# O sigui: en aquest cas el NaN NO és una dada perduda, sinó una informació implícita.
# Per això, en lloc d'imputar una data inventada, ho transformo en variables binàries i derivades.

# 3.1 Has_lapse: 1 si hi ha data de baixa (no NaN), 0 si no (NaN)
df_prep["Has_lapse"] = np.where(df_prep["Date_lapse"].isna(), 0, 1)

# 3.2 Is_active: el complementari (si hi ha lapse, ja no està activa)
# Si Has_lapse = 1 (cancel·lada) → Is_active = 0
# Si Has_lapse = 0 (sense baixa) → Is_active = 1
df_prep["Is_active"] = 1 - df_prep["Has_lapse"]

# 3.3 Conversió robusta de Date_lapse i Date_start_contract a datetime
# Ho torno a convertir per assegurar-me que realment estan en format datetime abans de fer diferències.
# Si ja venien correctes del fitxer net, no passa res.
df_prep["Date_lapse"] = pd.to_datetime(
    df_prep["Date_lapse"], format="%Y-%m-%d", errors="coerce"
)
df_prep["Date_start_contract"] = pd.to_datetime(
    df_prep["Date_start_contract"], format="%Y-%m-%d", errors="coerce"
)

# 3.4 Policy_duration: només per contractes finalitzats (Has_lapse=1)
# Calculo la durada (en anys) entre l'inici i la data de lapse.
# Si la pòlissa no té lapse, ho deixo com NaN perquè segueix sent nul estructural.
df_prep["Policy_duration"] = np.where(
    df_prep["Has_lapse"] == 1,
    (df_prep["Date_lapse"] - df_prep["Date_start_contract"]).dt.days / 365.25,
    np.nan
).round(2)  # Ho arrodoneixo a 2 decimals perquè sigui més llegible i suficient per aquesta fase

# Validació ràpida del lapse
# Miro la distribució i el percentatge de cancel·lacions per entendre el pes real d'aquest fenomen.
print("\nDistribució Has_lapse:")
display(df_prep["Has_lapse"].value_counts())

print(
    "Percentatge pòlisses cancel·lades:",
    round(df_prep["Has_lapse"].mean() * 100, 2), "%"
)

# ------------------------------------------------------------
# 4) Tractament de nuls baixos (Type_fuel)
# ------------------------------------------------------------
# Aquí el percentatge de nuls és baix, així que faig una imputació simple:
# els converteixo en una categoria "Unknown" per no perdre registres.
# (La codificació de categories la faré més endavant.)
if "Type_fuel" in df_prep.columns:
    # Compto nuls abans per tenir control de què estic tocant.
    n_fuel_nulls_before = df_prep["Type_fuel"].isna().sum()

    # Substitueixo NaN per "Unknown".
    df_prep["Type_fuel"] = df_prep["Type_fuel"].fillna("Unknown")

    # Verifico que ja no en queden.
    n_fuel_nulls_after = df_prep["Type_fuel"].isna().sum()

    print("\nType_fuel nuls abans:", n_fuel_nulls_before)
    print("Type_fuel nuls després:", n_fuel_nulls_after)

    # Miro la distribució (en percentatge) per veure quin pes real té "Unknown".
    print("Distribució Type_fuel:")
    display(df_prep["Type_fuel"].value_counts(normalize=True).rename("pct"))

# ------------------------------------------------------------
# 5) Tractament de nuls moderats (Length)
# ------------------------------------------------------------
# Aquí ja hi ha un percentatge notable de nuls, així que faig imputació controlada.
# Estratègia:
#   - Marco amb un flag quins valors he imputat (per transparència i traçabilitat).
#   - Imputo amb la mediana de Length dins de cada Type_risk.
#   - Si per algun segment no hi ha mediana (perquè tot és nul), faig servir la mediana global.
if "Length" in df_prep.columns:
    # 5.1 Creo un flag per saber exactament quines files han estat imputades.
    df_prep["Length_missing_flag"] = np.where(df_prep["Length"].isna(), 1, 0)

    n_len_nulls_before = df_prep["Length"].isna().sum()
    print("\nLength nuls abans:", n_len_nulls_before)

    # 5.2 Calculo la mediana per segment de risc.
    median_by_risk = df_prep.groupby("Type_risk")["Length"].median()

    # I també la mediana global per si em trobo segments sense valors útils.
    global_median = df_prep["Length"].median()

    # 5.3 Funció d’imputació segmentada fila a fila
    # Ho faig així perquè vull aplicar la lògica "mediana per segment" amb fallback global.
    def impute_length(row):
        # Només imputo si el valor original és NaN.
        if pd.isna(row["Length"]):
            # Agafo la mediana del segment corresponent.
            seg_med = median_by_risk.get(row["Type_risk"], np.nan)
            # Si el segment no té mediana, tiro de la global.
            return seg_med if not pd.isna(seg_med) else global_median
        # Si no era NaN, retorno el valor original.
        return row["Length"]

    # Aplico la funció a tot el dataframe.
    df_prep["Length"] = df_prep.apply(impute_length, axis=1)

    # Verifico que ja no queden nuls a Length.
    n_len_nulls_after = df_prep["Length"].isna().sum()
    print("Length nuls després:", n_len_nulls_after)

    # Miro descriptives per assegurar-me que la imputació no ha generat valors estranys.
    print("\nDescriptives Length després de la imputació:")
    display(df_prep["Length"].describe())

    print(
        "Percentatge imputat (Length_missing_flag=1):",
        round(df_prep["Length_missing_flag"].mean() * 100, 2), "%"
    )

# ------------------------------------------------------------
# 6) Comprovació final de nuls no estructurals
# ------------------------------------------------------------
# Recalculo la taula de nuls per assegurar-me que:
#   - només queden nuls que jo accepto com a estructurals
#   - no m’he deixat cap variable amb nuls "pendents" sense tractar

missing_count_after = df_prep.isna().sum()
missing_pct_after = (missing_count_after / len(df_prep) * 100).round(2)

missing_table_after = (
    pd.DataFrame({
        "missing_count": missing_count_after,
        "missing_pct": missing_pct_after
    })
    .sort_values("missing_count", ascending=False)
)

print("\nTaula de nuls FINAL (top 10):")
display(missing_table_after.head(10))

# Defineixo explícitament quines variables poden mantenir nuls perquè tenen sentit (estructurals).
structural_allowed = ["Date_lapse", "Policy_duration"]

# Em quedo amb les variables que NO haurien de tenir nuls (i hi busco problemes).
non_struct_nulls = missing_table_after.drop(index=structural_allowed, errors="ignore")

# Validació final:
# Si queda algun nul en variables no estructurals, prefereixo que peti aquí i no arrossegui el problema.
assert non_struct_nulls["missing_count"].sum() == 0, \
       "Hi ha nuls no estructurals pendents després del tractament!"

print("\nValidació correcta: només queden nuls estructurals esperats.")
print("Dimensions finals df_prep:", df_prep.shape)


clean_motor_insurance.csv carregat correctament
Dimensions: (105555, 34)

Duplicats exactes: 0

Taula de nuls inicial (top 10):


,missing_count,missing_pct
Date_lapse,70408,66.70
Length,10329,9.79
Type_fuel,1764,1.67
ID,0,0.00
Cylinder_capacity,0,0.00
Area,0,0.00
Second_driver,0,0.00
Year_matriculation,0,0.00
Power,0,0.00
N_doors,0,0.00



Distribució Has_lapse:


Has_lapse
0    70408
1    35147
Name: count, dtype: int64

Percentatge pòlisses cancel·lades: 33.3 %

Type_fuel nuls abans: 1764
Type_fuel nuls després: 0
Distribució Type_fuel:


...


Length nuls abans: 10329
Length nuls després: 0

Descriptives Length després de la imputació:


count    105555.000000
mean          4.185750
std           0.455467
min           1.978000
25%           3.941000
50%           4.202000
75%           4.433000
max           8.218000
Name: Length, dtype: float64

Percentatge imputat (Length_missing_flag=1): 9.79 %

Taula de nuls FINAL (top 10):


,missing_count,missing_pct
Policy_duration,70408,66.7
Date_lapse,70408,66.7
ID,0,0.0
Length,0,0.0
Year_matriculation,0,0.0
Power,0,0.0
Cylinder_capacity,0,0.0
Value_vehicle,0,0.0
N_doors,0,0.0
Type_fuel,0,0.0



Validació correcta: només queden nuls estructurals esperats.
Dimensions finals df_prep: (105555, 38)


#### <b>3.3.2.2 Derivació de variables temporals i coherència cronològica</b>

In [6]:
# ============================================================
# 3.3.2.2 Derivació de variables temporals i coherència cronològica
# ============================================================
# En aquest script derivo i valido magnituds temporals clau a partir de
# pòlisses d’assegurança motor que ja tinc pre-processades a `df_prep`.
# Aquí NO faig cap correcció automàtica: només genero càlculs, flags i
# un petit report per veure si hi ha incoherències cronològiques.
# ============================================================

import pandas as pd  # Importo pandas per treballar amb dataframes i, sobretot aquí, amb dates
import numpy as np   # Importo numpy perquè em va bé per a lògica condicional i tractament numèric

# Copio el dataframe per treballar tranquil sense tocar l'original (`df_prep`).
# Així puc fer comprovacions i càlculs sense risc d'arrossegar canvis a altres blocs.
df_time = df_prep.copy()

# ------------------------------------------------------------
# 0) FORÇAR CAST DE TOTES LES DATES A DATETIME
# ------------------------------------------------------------
# Abans de derivar res, m'asseguro que totes les columnes de data estan en datetime64.
# Encara que a la fase anterior ja estiguin convertides, aquí ho torno a forçar per coherència
# i per evitar que algun tipus "object" em trenqui els càlculs de diferències.

date_cols = [
    "Date_start_contract",   # Inici del contracte/pòlissa
    "Date_last_renewal",     # Data de l’última renovació registrada
    "Date_next_renewal",     # Data prevista per a la renovació futura
    "Date_birth",            # Data de naixement del conductor/titular
    "Date_driving_licence",  # Data d'obtenció del carnet de conduir
    "Date_lapse"             # Data de baixa/lapse (si existeix; si no, serà NaT)
]

for c in date_cols:
    # Com que espero dates en format ISO (YYYY-MM-DD), especifico el format explícit.
    # Deixo errors="coerce" perquè qualsevol valor no parsejable passi a NaT i el pugui detectar després.
    df_time[c] = pd.to_datetime(
        df_time[c],
        format="%Y-%m-%d",
        errors="coerce"
    )

# Reviso els tipus resultants per assegurar-me que m'ha quedat tot com toca.
print("Tipus després de la conversió forçada:")
print(df_time[date_cols].dtypes)

# ------------------------------------------------------------
# 1) VARIABLES TEMPORALS DERIVADES
# ------------------------------------------------------------
# Un cop les dates són datetime, derivo variables en format numèric que són més útils per a models
# i també per controlar incoherències (per exemple, antiguitats negatives).

# Edat del conductor (anys) a l'inici del contracte
df_time["Driver_age"] = (
    df_time["Date_start_contract"] - df_time["Date_birth"]
).dt.days / 365.25
# Ho passo a anys dividint per 365.25 perquè és una aproximació raonable (inclou anys de traspàs).

# Antiguitat del carnet (anys) a la darrera renovació
df_time["Licence_age"] = (
    df_time["Date_last_renewal"] - df_time["Date_driving_licence"]
).dt.days / 365.25
# Això em dona quants anys fa que té el carnet quan arriba a la darrera renovació.

# Antiguitat de la pòlissa (anys) fins la darrera renovació
df_time["Policy_age"] = (
    df_time["Date_last_renewal"] - df_time["Date_start_contract"]
).dt.days / 365.25
# Aquí espero valors positius; si em surten negatius és que l'ordre temporal no quadra.

# Dies fins la pròxima renovació
df_time["Days_to_next"] = (
    df_time["Date_next_renewal"] - df_time["Date_last_renewal"]
).dt.days
# Ho deixo en dies perquè és un indicador molt directe (i perquè em facilita detectar valors negatius).

# Antiguitat del vehicle en anys
df_time["Vehicle_age"] = (
    df_time["Date_start_contract"].dt.year - df_time["Year_matriculation"]
)
# Faig aquesta derivada en anys "calendari" perquè aquí només necessito una aproximació senzilla
# (any d'inici de contracte menys any de matriculació).

# ------------------------------------------------------------
# 2) DETECTAR ANOMALIES TEMPORALS
# ------------------------------------------------------------
# Ara compto incoherències cronològiques i casos implausibles.
# No els arreglo: només vull quantificar-los per saber si tinc problemes seriosos de qualitat de dades.

anomalies = {
    "birth_after_licence": (df_time["Date_birth"] > df_time["Date_driving_licence"]).sum(),
    # Naixement posterior a l'obtenció del carnet → això és impossible, així que aquí hi ha error segur.

    "licence_negative": (df_time["Licence_age"] < 0).sum(),
    # Antiguitat del carnet negativa → vol dir que la "darrera renovació" seria abans que la data del carnet.

    "policy_negative": (df_time["Policy_age"] < 0).sum(),
    # Antiguitat de la pòlissa negativa → la renovació seria anterior a l'inici del contracte (inconsistència forta).

    "next_before_last": (df_time["Days_to_next"] < 0).sum(),
    # La renovació futura és anterior a la darrera → cronologia invertida.

    "lapse_before_start": (
        df_time["Date_lapse"].notna() &
        (df_time["Date_lapse"] < df_time["Date_start_contract"])
    ).sum(),
    # Hi ha lapse però cau abans que comenci el contracte → inconsistent si 'lapse' és baixa/cancel·lació real.

    "lapse_before_last": (
        df_time["Date_lapse"].notna() &
        (df_time["Date_lapse"] < df_time["Date_last_renewal"])
    ).sum()
    # Lapse abans de la darrera renovació → pot ser error, o pot dependre de com estigui definit al negoci.
}

# Imprimeixo el resum d'anomalies perquè em quedi un report ràpid per a la memòria.
print("\n--- ANOMALIES DETECTADES ---")
for k, v in anomalies.items():
    print(f"{k}: {v}")

# ------------------------------------------------------------
# 3) DESCRIPTIVES DE LES VARIABLES TEMPORALS
# ------------------------------------------------------------
# Miro estadístiques bàsiques per veure rangs, valors extrems i si hi ha números estranys.
# Això em serveix també per detectar edats impossibles (massa altes o negatives), vehicles amb edat negativa, etc.

temp_vars = ["Driver_age", "Licence_age", "Policy_age", "Vehicle_age", "Days_to_next"]

print("\n--- DESCRIPTIVES VARIABLES TEMPORALS ---")
display(df_time[temp_vars].describe().T)

# ------------------------------------------------------------
# 4) PREVIEW FINAL
# ------------------------------------------------------------
# Faig una inspecció visual de les primeres files amb les dates i derivades per assegurar-me que tot té sentit.

print("\nMostra de registres amb variables temporals derivades:")
display(df_time[[
    "Date_start_contract", "Date_last_renewal", "Date_next_renewal",
    "Date_birth", "Date_driving_licence", "Date_lapse",
    "Driver_age", "Licence_age", "Policy_age", "Vehicle_age", "Days_to_next"
]].head())


Tipus després de la conversió forçada:
Date_start_contract     datetime64[ns]
Date_last_renewal       datetime64[ns]
Date_next_renewal       datetime64[ns]
Date_birth              datetime64[ns]
Date_driving_licence    datetime64[ns]
Date_lapse              datetime64[ns]
dtype: object

--- ANOMALIES DETECTADES ---
birth_after_licence: 0
licence_negative: 31
policy_negative: 68
next_before_last: 0
lapse_before_start: 0
lapse_before_last: 404

--- DESCRIPTIVES VARIABLES TEMPORALS ---


,count,mean,std,min,25%,50%,75%,max
Driver_age,105555.0,44.344420,12.765212,18.015058,34.234086,43.972621,53.733060,91.529090
Licence_age,105555.0,24.765484,12.486053,-2.242300,14.410678,23.854894,34.171116,74.028747
Policy_age,105555.0,2.981346,3.923427,-0.911704,0.000000,1.002053,4.000000,37.998631
Vehicle_age,105555.0,9.244887,7.148871,0.000000,4.000000,9.000000,13.000000,68.000000
Days_to_next,105555.0,365.088020,0.283326,365.000000,365.000000,365.000000,365.000000,366.000000



Mostra de registres amb variables temporals derivades:


,Date_start_contract,Date_last_renewal,Date_next_renewal,Date_birth,Date_driving_licence,Date_lapse,Driver_age,Licence_age,Policy_age,Vehicle_age,Days_to_next
0,2015-11-05,2015-11-05,2016-11-05,1956-04-15,1976-03-20,NaT,59.556468,39.627652,0.000000,11,366
1,2015-11-05,2016-11-05,2017-11-05,1956-04-15,1976-03-20,NaT,59.556468,40.629706,1.002053,11,365
2,2015-11-05,2017-11-05,2018-11-05,1956-04-15,1976-03-20,NaT,59.556468,41.629021,2.001369,11,365
3,2015-11-05,2018-11-05,2019-11-05,1956-04-15,1976-03-20,NaT,59.556468,42.628337,3.000684,11,365
4,2017-09-26,2017-09-26,2018-09-26,1956-04-15,1976-03-20,NaT,61.448323,41.519507,0.000000,13,365


### <b>3.3.2.3 Tractament d'anomalies temporals i incoherències</b>

In [8]:
# ============================================================
# 3.3.2.3 Tractament d'anomalies temporals i incoherències
# ============================================================
# Objectiu:
#   - Detectar i sanejar incoherències temporals residuals que em queden
#     després de la fase anterior (df_time).
#   - Crear flags de traçabilitat per tipologia d’anomalia, perquè vull
#     poder identificar exactament quins registres he hagut de tocar.
#   - Aplicar correccions conservadores (sense inventar informació nova):
#     aquí no "reconstrueixo" dates, simplement les retallo perquè no
#     generin valors impossibles o incoherents.
#        * Licence_age < 0:
#             -> ajusto Date_driving_licence = Date_last_renewal
#        * Policy_age  < 0:
#             -> ajusto Date_start_contract = Date_last_renewal
#        * Date_lapse < Date_last_renewal (si hi ha lapse):
#             -> ajusto Date_lapse = Date_last_renewal
#   - Recalcular totes les derivades temporals afectades després de canviar dates.
#   - Validar que ja no queden antiguitats negatives ni incoherències de lapse.
#
# Entrada:
#   df_time = dataset amb derivades temporals i anomalies detectades
#
# Sortida:
#   df_time_corr = dataset temporalment coherent + flags de control
# ============================================================

# Fem una còpia per no modificar df_time original
# Ho faig així perquè vull mantenir df_time com a "foto" del resultat previ,
# i treballar la correcció en un objecte separat.
df_time_corr = df_time.copy()

# ------------------------------------------------------------
# 1) Flags de traçabilitat (abans de corregir)
# ------------------------------------------------------------
# En aquesta secció només marco quins registres són incoherents, sense tocar res.
# Això em serveix per dues coses:
#   - Saber quants casos problemàtics tinc de cada tipus.
#   - Deixar constància (flags) de què he corregit després.
#
# 1.1 Llicència incoherent: Licence_age < 0
#     (equival a Date_driving_licence posterior a Date_last_renewal)
df_time_corr["Licence_incoherent_flag"] = np.where(
    df_time_corr["Licence_age"] < 0,  # condició d'incoherència
    1,                                 # valor si condició certa
    0                                  # valor si condició falsa
)

# 1.2 Pòlissa incoherent: Policy_age < 0
#     (equival a Date_start_contract posterior a Date_last_renewal)
df_time_corr["Policy_incoherent_flag"] = np.where(
    df_time_corr["Policy_age"] < 0,
    1,
    0
)

# 1.3 Lapse incoherent: Date_lapse abans de Date_last_renewal, però només
#     en registres on realment hi ha lapse (Date_lapse no és NaN).
df_time_corr["Lapse_incoherent_flag"] = np.where(
    (df_time_corr["Date_lapse"].notna()) &  # hi ha lapse
    (df_time_corr["Date_lapse"] < df_time_corr["Date_last_renewal"]),  # és abans de last_renewal
    1,
    0
)

print("Flags d’incoherència (abans de corregir):")
# Sumo els flags per veure ràpidament quants casos hi ha de cada tipus d'anomalia.
display(df_time_corr[
    ["Licence_incoherent_flag", "Policy_incoherent_flag", "Lapse_incoherent_flag"]
].sum())

# ------------------------------------------------------------
# 2) Correccions conservadores de dates
# ------------------------------------------------------------
# Un cop ja tinc les flags guardades, aplico les correccions mínimes.
# La idea és simple: si una data "va més enllà" del que és coherent,
# la porto (retallo) fins a la data de referència `Date_last_renewal`.
# Així evito valors negatius, però no em poso a inventar informació nova.

# 2.1 Correcció Licence_age negativa:
#     Si Licence_incoherent_flag = 1 vol dir que Licence_age < 0.
#     Ajusto Date_driving_licence = Date_last_renewal
#     ⇒ Després d'això, Licence_age passa a ser 0.
mask_lic = df_time_corr["Licence_incoherent_flag"] == 1
df_time_corr.loc[mask_lic, "Date_driving_licence"] = df_time_corr.loc[mask_lic, "Date_last_renewal"]

# 2.2 Correcció Policy_age negativa:
#     Si Policy_incoherent_flag = 1 vol dir que Policy_age < 0.
#     Ajusto Date_start_contract = Date_last_renewal
#     ⇒ Policy_age també queda a 0.
mask_pol = df_time_corr["Policy_incoherent_flag"] == 1
df_time_corr.loc[mask_pol, "Date_start_contract"] = df_time_corr.loc[mask_pol, "Date_last_renewal"]

# 2.3 Correcció lapse incoherent:
#     Si Lapse_incoherent_flag = 1 vol dir que Date_lapse < Date_last_renewal.
#     Ajusto Date_lapse = Date_last_renewal per garantir que la baixa no quedi
#     abans de l’última renovació registrada.
mask_lap = df_time_corr["Lapse_incoherent_flag"] == 1
df_time_corr.loc[mask_lap, "Date_lapse"] = df_time_corr.loc[mask_lap, "Date_last_renewal"]

# ------------------------------------------------------------
# 3) Recalcular derivades temporals després de corregir
# ------------------------------------------------------------
# Com que he modificat dates, recalculo totes les derivades que en depenen.
# Si no ho fes, em quedaria amb variables temporals "desfasades" respecte a les dates corregides.

# Edat del conductor a l'inici del contracte
df_time_corr["Driver_age"] = (
    (df_time_corr["Date_start_contract"] - df_time_corr["Date_birth"]).dt.days / 365.25
)

# Antiguitat del carnet a la darrera renovació
df_time_corr["Licence_age"] = (
    (df_time_corr["Date_last_renewal"] - df_time_corr["Date_driving_licence"]).dt.days / 365.25
)

# Antiguitat de la pòlissa a la darrera renovació
df_time_corr["Policy_age"] = (
    (df_time_corr["Date_last_renewal"] - df_time_corr["Date_start_contract"]).dt.days / 365.25
)

# Dies fins la pròxima renovació
df_time_corr["Days_to_next"] = (
    (df_time_corr["Date_next_renewal"] - df_time_corr["Date_last_renewal"]).dt.days
)

# Antiguitat del vehicle a l'inici del contracte
df_time_corr["Vehicle_age"] = (
    df_time_corr["Date_start_contract"].dt.year - df_time_corr["Year_matriculation"]
)

# Si Policy_duration depèn de Date_start_contract i Date_lapse, també la recalculo
# (i la mantinc com NaN quan és un nul estructural).
df_time_corr["Policy_duration"] = np.where(
    df_time_corr["Has_lapse"] == 1,  # només té sentit per pòlisses cancel·lades
    (df_time_corr["Date_lapse"] - df_time_corr["Date_start_contract"]).dt.days / 365.25,
    np.nan
).round(2)

# ------------------------------------------------------------
# 4) Validació post-correcció
# ------------------------------------------------------------
# Torno a comptar incoherències per assegurar-me que les correccions han funcionat.
# Aquí vull que:
#   - Licence_age no sigui negativa
#   - Policy_age no sigui negativa
#   - i que cap lapse quedi abans de la darrera renovació

lic_neg_after = (df_time_corr["Licence_age"] < 0).sum()
pol_neg_after = (df_time_corr["Policy_age"] < 0).sum()
lap_incoh_after = (
    (df_time_corr["Date_lapse"].notna()) &
    (df_time_corr["Date_lapse"] < df_time_corr["Date_last_renewal"])
).sum()

print("\n--- VALIDACIÓ POST-CORRECCIÓ ---")
print("Licence_age < 0 :", lic_neg_after)
print("Policy_age  < 0 :", pol_neg_after)
print("Lapse incoherent :", lap_incoh_after)

# ------------------------------------------------------------
# 5) Mostra de registres corregits (opcional)
# ------------------------------------------------------------
# Per fer una revisió manual ràpida, mostro una petita mostra de registres
# que tenien algun tipus d'incoherència (llicència, pòlissa o lapse).
print("\nMostra de registres corregits (Licence/Policy/Lapse):")
display(df_time_corr.loc[
    mask_lic | mask_pol | mask_lap,
    [
        "ID",                     # identificador del registre/pòlissa
        "Date_start_contract",    # data d'inici (pot haver estat retallada)
        "Date_last_renewal",      # data de referència utilitzada per corregir
        "Date_lapse",             # data de baixa (pot haver estat retallada)
        "Date_driving_licence",   # data del carnet (pot haver estat retallada)
        "Driver_age",             # derivades recalculades després de la correcció
        "Licence_age",
        "Policy_age",
        "Licence_incoherent_flag",  # flags per saber què estava incoherent
        "Policy_incoherent_flag",
        "Lapse_incoherent_flag"
    ]
].head())

# df_time_corr em queda llest per continuar amb els següents passos:
#   - codificació categòrica bàsica
#   - derivació de variables de negoci
#   - i, si cal, exportació final (p. ex. transformed_motor_insurance.csv)


Flags d’incoherència (abans de corregir):


Licence_incoherent_flag     31
Policy_incoherent_flag      68
Lapse_incoherent_flag      404
dtype: int64


--- VALIDACIÓ POST-CORRECCIÓ ---
Licence_age < 0 : 0
Policy_age  < 0 : 0
Lapse incoherent : 0

Mostra de registres corregits (Licence/Policy/Lapse):


,ID,Date_start_contract,Date_last_renewal,Date_lapse,Date_driving_licence,Driver_age,Licence_age,Policy_age,Licence_incoherent_flag,Policy_incoherent_flag,Lapse_incoherent_flag
268,136,2016-04-13,2017-04-13,2017-04-13,1990-04-26,45.147159,26.965092,0.999316,0,0,1
350,180,2015-09-15,2016-09-15,2016-09-15,1986-05-30,48.227242,30.297057,1.002053,0,0,1
371,192,2011-06-17,2016-06-17,2016-06-17,1994-12-01,53.127995,21.544148,5.002053,0,0,1
1454,726,2007-06-30,2017-06-30,2017-06-30,1990-01-01,53.801506,27.493498,10.001369,0,0,1
1729,861,2018-01-15,2018-01-15,NaT,2007-07-11,47.047228,10.516085,0.000000,0,1,0


### <b>3.3.2.4 Variables de negoci derivades</b>

In [10]:
# ============================================================
# 3.3.2.4 Variables de negoci derivades
# ============================================================
# Objectiu:
#   - Derivar variables orientades a la perspectiva actuarial i econòmica,
#     pensades per capturar sinistralitat i rendibilitat tècnica.
#   - Afegir indicadors binaris tant de sinistre anual com d’històric.
#   - Calcular la ràtio cost/prima, que és una mesura clau per analitzar
#     si una pòlissa és rendible o deficitària.
#   - Fer una validació bàsica de distribucions i coherència de les noves variables.
#
# Input esperat:
#   df_time_corr → dataset amb les dates ja coherents i derivades temporals revisades.
#
# Output:
#   df_business → dataset ampliat amb variables de negoci derivades.
# ============================================================

# Faig una còpia del dataset corregit per no modificar df_time_corr directament.
# Així mantinc una separació clara entre la fase temporal i la fase de negoci.
df_business = df_time_corr.copy()

# ------------------------------------------------------------
# 1) Indicador de sinistre anual
# ------------------------------------------------------------
# A partir del nombre de sinistres de l’any (`N_claims_year`),
# creo una variable binària molt útil per models:
#   - 1 si hi ha almenys un sinistre durant l’any
#   - 0 si no n’hi ha cap
# Qualsevol valor > 0 el considero sinistralitat anual.
df_business["Has_claims_year"] = (df_business["N_claims_year"] > 0).astype(int)

# ------------------------------------------------------------
# 2) Indicador d’històric de sinistres
# ------------------------------------------------------------
# De manera anàloga, creo un indicador d’històric de sinistres a partir
# de `N_claims_history`:
#   - 1 si al llarg de l’històric hi ha hagut algun sinistre
#   - 0 si no hi ha cap sinistre registrat
# Això em permet diferenciar conductors “nets” de conductors amb historial.
df_business["Has_claims_history"] = (df_business["N_claims_history"] > 0).astype(int)

# ------------------------------------------------------------
# 3) Ràtio econòmica cost/prima
# ------------------------------------------------------------
# Aquí calculo una de les variables clau des del punt de vista actuarial:
# la ràtio entre el cost de sinistres anuals i la prima anual.
#
# Interpretació:
#   - Ràtio > 1  → el cost de sinistres supera la prima (pòlissa deficitària).
#   - Ràtio < 1  → la prima cobreix els sinistres (marge tècnic positiu).
#   - Si Premium = 0, la divisió genera infinits, que prefereixo convertir a NaN.
df_business["Claims_to_premium_ratio"] = (
    df_business["Cost_claims_year"] / df_business["Premium"]
).replace([np.inf, -np.inf], np.nan)  # Evito infinits quan la prima és zero

# ------------------------------------------------------------
# 4) Validació bàsica de les noves variables
# ------------------------------------------------------------
# Faig comprovacions senzilles per assegurar-me que les variables tenen sentit
# i que no hi ha distribucions estranyes.

print("Distribució Has_claims_year:")
# value_counts() em permet veure quants casos amb i sense sinistre anual hi ha.
print(df_business["Has_claims_year"].value_counts())

print("\nDistribució Has_claims_history:")
# Mateixa idea, però per a l’històric de sinistres.
print(df_business["Has_claims_history"].value_counts())

print("\nDescriptives Claims_to_premium_ratio:")
# describe() em dona una visió ràpida de rangs, mitjana, quartils i valors extrems.
print(df_business["Claims_to_premium_ratio"].describe())

# ------------------------------------------------------------
# 5) Mostra de registres amb variables de negoci derivades
# ------------------------------------------------------------
# Mostro una petita mostra de files amb:
#   - variables originals de sinistres
#   - la prima
#   - i les noves variables de negoci
# per fer una inspecció visual ràpida i assegurar-me que tot quadra.
print("\nMostra de registres amb variables de negoci derivades:")
print(df_business[[
    "N_claims_year", "Has_claims_year",
    "N_claims_history", "Has_claims_history",
    "Premium", "Cost_claims_year", "Claims_to_premium_ratio"
]].head())


Distribució Has_claims_year:
Has_claims_year
0    85909
1    19646
Name: count, dtype: int64

Distribució Has_claims_history:
Has_claims_history
1    68959
0    36596
Name: count, dtype: int64

Descriptives Claims_to_premium_ratio:
count    105555.000000
mean          0.476172
std           4.720622
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max        1071.248039
Name: Claims_to_premium_ratio, dtype: float64

Mostra de registres amb variables de negoci derivades:
   N_claims_year  Has_claims_year  N_claims_history  Has_claims_history  \
0              0                0                 0                   0   
1              0                0                 0                   0   
2              0                0                 0                   0   
3              0                0                 0                   0   
4              0                0                 0                   0   

   Premium  Cost_claims_year  C

### <b>3.3.2.5 Validació final, exportació i mapa de features</b>

In [12]:
### ============================================================
# 3.3.2.5 Validació final, exportació i mapa de features
# ============================================================
# En aquest últim script tanco la fase 3.3.2 deixant el dataset preparat
# per a modelització/consum extern, i em genero també documentació bàsica.
#
# Dataset d’entrada:
#   df_business → dataset final amb:
#       - nuls estructurals únicament a Date_lapse / Policy_duration
#       - variables temporals derivades
#       - incoherències corregides + flags de control
#       - variables de negoci derivades
#
# Sortides que em genero:
#   * transformed_motor_insurance.csv  → dataset final ordenat i exportat
#   * schema_after_3_3_2.csv           → esquema amb dtypes per traçabilitat
#   * Diccionari intern de mapa de features (feature_map) → llista de variables per família de models
# ============================================================

# Faig una còpia per no tocar df_business directament.
# Prefereixo treballar aquí sobre df_final perquè aquesta és la "foto final" abans d'exportar.
df_final = df_business.copy()

# ------------------------------------------------------------
# 1) VALIDACIÓ FINAL DE NULS
# ------------------------------------------------------------
# Aquí faig l’última comprovació de qualitat: vull assegurar-me que
# ja no queden nuls no estructurals abans d'exportar el dataset.

# Calculo quants nuls hi ha per variable al dataset final.
missing_table_final = df_final.isna().sum().to_frame("missing_count")
missing_table_final["missing_pct"] = (
    missing_table_final["missing_count"] / len(df_final) * 100
).round(2)

print("\n--- Nuls finals (top 10) ---")
# Mostro les 10 variables amb més nuls per fer una inspecció visual ràpida.
display(missing_table_final.sort_values("missing_count", ascending=False).head(10))

# Defineixo explícitament quins nuls accepto perquè són estructurals (per definició de negoci):
#   - Date_lapse: només té valor quan hi ha cancel·lació (Has_lapse = 1).
#   - Policy_duration: només es pot calcular si Has_lapse = 1.
structural_allowed = ["Date_lapse", "Policy_duration"]

# Elimino aquestes variables de l'anàlisi per centrar-me només en la resta.
non_structural_nulls = missing_table_final.drop(index=structural_allowed, errors="ignore")

# Validació estricta:
# Si aquí queda algun nul, prefereixo que peti i no exportar un dataset "brut".
assert non_structural_nulls["missing_count"].sum() == 0, \
       "Encara hi ha nuls no estructurals!"
print("\nValidació correcta: només queden nuls estructurals esperats.")

# ------------------------------------------------------------
# 2) ORDENACIÓ LÒGICA DE VARIABLES PER FAMÍLIES
# ------------------------------------------------------------
# Ara reordeno les columnes en famílies conceptuals perquè:
#   - sigui més fàcil llegir i debugar el dataset
#   - la modelització posterior sigui més clara (sobretot quan exporto i torno a carregar)
#   - tingui una estructura consistent a la memòria/documentació

# Identificador únic del registre/pòlissa
cols_id = ["ID"]

# Dates “crues” / originals del negoci
cols_dates = [
    "Date_start_contract", "Date_last_renewal", "Date_next_renewal",
    "Date_birth", "Date_driving_licence", "Date_lapse"
]

# Variables temporals derivades (antiguitats, durades, etc.)
cols_temporal_der = [
    "Driver_age", "Licence_age", "Vehicle_age", "Policy_age",
    "Days_to_next", "Policy_duration"
]

# Flags de qualitat i d’incoherències corregides
cols_flags = [
    "Has_lapse",
    "Licence_incoherent_flag", "Policy_incoherent_flag", "Lapse_incoherent_flag",
    "Length_missing_flag"
]

# Característiques físiques / tècniques del vehicle
cols_vehicle = [
    "Year_matriculation", "Power", "Cylinder_capacity",
    "Value_vehicle", "N_doors", "Length", "Weight"
]

# Variables de risc i perfil comercial
cols_risk = [
    "Distribution_channel", "Seniority", "Policies_in_force",
    "Max_policies", "Max_products", "Type_risk", "Area", "Second_driver"
]

# Variables de negoci i sinistralitat
cols_business = [
    "Premium", "N_claims_year", "Cost_claims_year",
    "N_claims_history", "R_Claims_history",
    "Has_claims_year", "Has_claims_history", "Claims_to_premium_ratio",
    "Payment", "Lapse", "Type_fuel"
]

# Ajunto totes les columnes en l’ordre que vull per al dataset final.
all_cols = (
    cols_id
    + cols_dates
    + cols_temporal_der
    + cols_flags
    + cols_vehicle
    + cols_risk
    + cols_business
)

# Abans de reordenar, comprovo que totes les columnes esperades existeixen.
# Això m'ajuda a detectar ràpid:
#   - canvis de nom
#   - passos previs que no s'han executat
#   - o variables que no estaven al dataset original
missing_in_final = [c for c in all_cols if c not in df_final.columns]
if missing_in_final:
    # Si falta alguna columna, paro aquí i indico exactament quines.
    raise ValueError(f"Columnes esperades però absents a df_final: {missing_in_final}")

# Un cop validat, reordeno efectivament el dataframe.
df_final = df_final[all_cols]

print("\nReordenació de variables completada.")
print("Dimensions df_final:", df_final.shape)

# ------------------------------------------------------------
# 3) GENERACIÓ DE L’ESQUEMA FINAL (schema_after_3_3_2.csv)
# ------------------------------------------------------------
# Em genero un esquema simple per documentar el dataset:
# nom de variable + tipus (dtype) que li assigna pandas.
# Això m'anirà bé per traçabilitat i per detectar canvis quan torni a carregar fitxers.

schema = pd.DataFrame({
    "variable": df_final.columns,
    "dtype": df_final.dtypes.astype(str)
})

# Exporto l’esquema a CSV perquè quedi com a documentació externa.
schema.to_csv("schema_after_3_3_2.csv", index=False, encoding="utf-8")
print("\n📄 schema_after_3_3_2.csv generat correctament.")
display(schema.head(10))

# ------------------------------------------------------------
# 4) MAPA DE FEATURES PER FAMÍLIA DE MODELS
# ------------------------------------------------------------
# Aquí defineixo un "mapa" intern de features:
# una llista explícita de quines variables vull utilitzar a cada model.
# M'agrada deixar-ho en un diccionari perquè després és fàcil reutilitzar-lo o adaptar-lo.

feature_map = {
    # FREQÜÈNCIA (classificació Has_claims_year):
    # Variables que jo considero que poden explicar la probabilitat de tenir sinistre en l’any.
    "freq_model_features": [
        "Driver_age", "Licence_age", "Vehicle_age", "Has_lapse",
        "Policy_duration", "Second_driver", "Area", "Type_risk",
        "Type_fuel", "Has_claims_history", "Value_vehicle", "Power",
        "Premium", "Seniority", "Policies_in_force", "Distribution_channel"
    ],

    # SEVERITAT (regressió Cost_claims_year condicionada a tenir sinistre):
    # Aquí em centro més en variables que poden influir en el cost quan hi ha sinistre.
    "sev_model_features": [
        "Vehicle_age", "Value_vehicle", "Power", "Type_risk",
        "Area", "Policy_duration", "Claims_to_premium_ratio",
        "Weight", "Cylinder_capacity", "Length"
    ],

    # Flags de qualitat/traçabilitat:
    # No necessàriament entren al model, però els vull tenir controlats i disponibles per auditoria.
    "flags_features": cols_flags,

    # Variables objectiu (targets) dels models:
    #   - Freqüència: Has_claims_year (binària)
    #   - Severitat: Cost_claims_year (contínua)
    "targets": ["Has_claims_year", "Cost_claims_year"]
}

print("\n--- Mapa de features ---")
for k, v in feature_map.items():
    print(f"{k}: {v}")

# Converteixo el mapa a un DataFrame per veure'l millor i, si cal, guardar-lo o revisar-lo fàcilment.
mapa_features_df = pd.DataFrame(
    [(grp, var) for grp, vars_ in feature_map.items() for var in vars_],
    columns=["feature_group", "variable"]
)
display(mapa_features_df.head(20))

# ------------------------------------------------------------
# 5) EXPORTACIÓ DEL DATASET FINAL
# ------------------------------------------------------------
# Finalment exporto el dataset final, ja ordenat i validat, llest per modelització
# o per ser consumit per altres eines/processos.

df_final.to_csv("transformed_motor_insurance.csv", index=False, encoding="utf-8")

print("\n============================================================")
print("EXPORTACIÓ COMPLETADA")
print("Fitxers generats:")
print(" - transformed_motor_insurance.csv")
print(" - schema_after_3_3_2.csv")
print("============================================================")



--- Nuls finals (top 10) ---


,missing_count,missing_pct
Date_lapse,70408,66.7
Policy_duration,70408,66.7
ID,0,0.0
Value_vehicle,0,0.0
N_doors,0,0.0
Type_fuel,0,0.0
Length,0,0.0
Weight,0,0.0
Driver_age,0,0.0
Licence_age,0,0.0



Validació correcta: només queden nuls estructurals esperats.

Reordenació de variables completada.
Dimensions df_final: (105555, 44)

📄 schema_after_3_3_2.csv generat correctament.


,variable,dtype
ID,ID,int64
Date_start_contract,Date_start_contract,datetime64[ns]
Date_last_renewal,Date_last_renewal,datetime64[ns]
Date_next_renewal,Date_next_renewal,datetime64[ns]
Date_birth,Date_birth,datetime64[ns]
Date_driving_licence,Date_driving_licence,datetime64[ns]
Date_lapse,Date_lapse,datetime64[ns]
Driver_age,Driver_age,float64
Licence_age,Licence_age,float64
Vehicle_age,Vehicle_age,int64



--- Mapa de features ---
freq_model_features: ['Driver_age', 'Licence_age', 'Vehicle_age', 'Has_lapse', 'Policy_duration', 'Second_driver', 'Area', 'Type_risk', 'Type_fuel', 'Has_claims_history', 'Value_vehicle', 'Power', 'Premium', 'Seniority', 'Policies_in_force', 'Distribution_channel']
sev_model_features: ['Vehicle_age', 'Value_vehicle', 'Power', 'Type_risk', 'Area', 'Policy_duration', 'Claims_to_premium_ratio', 'Weight', 'Cylinder_capacity', 'Length']
flags_features: ['Has_lapse', 'Licence_incoherent_flag', 'Policy_incoherent_flag', 'Lapse_incoherent_flag', 'Length_missing_flag']
targets: ['Has_claims_year', 'Cost_claims_year']


,feature_group,variable
0,freq_model_features,Driver_age
1,freq_model_features,Licence_age
2,freq_model_features,Vehicle_age
3,freq_model_features,Has_lapse
4,freq_model_features,Policy_duration
5,freq_model_features,Second_driver
6,freq_model_features,Area
7,freq_model_features,Type_risk
8,freq_model_features,Type_fuel
9,freq_model_features,Has_claims_history



EXPORTACIÓ COMPLETADA
Fitxers generats:
 - transformed_motor_insurance.csv
 - schema_after_3_3_2.csv


#### <b>Rebaixar dataset a 4 decimals, decimals i milers</b>

In [14]:
# -------------------------------
# 1) Carregar dataset final
# -------------------------------
# Carrego el dataset final ja transformat, que és el que vull deixar
# preparat per a lliurament o ús extern (Excel, informes, etc.).
df = pd.read_csv("transformed_motor_insurance.csv")

# -------------------------------
# 2) Identificar columnes float
# -------------------------------
# Detecto automàticament totes les columnes de tipus float,
# ja que només vull aplicar arrodoniment i canvi de format a aquestes.
float_cols = df.select_dtypes(include="float").columns.tolist()

# -------------------------------
# 3) Arrodonir les float a 4 decimals
# -------------------------------
# Arrodoneixo totes les columnes numèriques float a 4 decimals.
# Això em dona un bon equilibri entre precisió i llegibilitat.
df[float_cols] = df[float_cols].round(4)

# -------------------------------
# 4) Convertir format numèric:
#    - sense separador de milers
#    - decimals amb coma ','
# -------------------------------
# Converteixo els valors float a text per adaptar-los al format europeu:
#   - elimino el punt decimal
#   - el substitueixo per coma
# Això és especialment útil si el CSV s'ha d'obrir amb Excel en configuració europea.
for col in float_cols:
    df[col] = (
        df[col]
        .astype(str)                          # Passo els valors a string
        .str.replace('.', ',', regex=False)   # Substitueixo el punt decimal per coma
    )

# -------------------------------
# 5) Exportar CSV final net
# -------------------------------
# Exporto el dataset final amb decimals europeus i arrodonits,
# llest per a lliurament o ús directe en eines ofimàtiques.
output_path = "transformed_motor_insurance_rounded_european.csv"
df.to_csv(output_path, index=False, encoding="utf-8")

# Retorno el path per confirmar quin fitxer s'ha generat
output_path


'transformed_motor_insurance_rounded_european.csv'